In [5]:
#this scrapes an individual puzzle and outputs in a csv

import csv
import json
from bs4 import BeautifulSoup

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8"/>
    <meta content="width=device-width" name="viewport"/>
    <title>Connections – Puzzle 274</title>
    <!-- Other meta tags -->
</head>
<body>
    <script id="__NEXT_DATA__" type="application/json">{"props":{"pageProps":{"id":"274","answers":[{"description":"TREAT WITH EXCESSIVE CARE","words":["BABY","MOTHER","PAMPER","SPOIL"],"emoji":"🟨","color":"#fbd400"},{"description":"BACKSIDE","words":["BOOTY","BUM","CAN","REAR"],"emoji":"🟩","color":"#69e352"},{"description":"THINGS IN A SPA LOCKER ROOM","words":["ROBE","SLIPPERS","TOWEL","WASHCLOTH"],"emoji":"🟦","color":"#5492ff"},{"description":"CAT’S ___","words":["CRADLE","EYE","MEOW","PAJAMAS"],"emoji":"🟪","color":"#df7bea"}],"isNYT":true},"__N_SSG":true},"page":"/nyt/[key]","query":{"key":"274"},"buildId":"ykRKfQq4_oL2Td9FRrTP3","isFallback":false,"gsp":true,"scriptLoader":[]}</script>
</body>
</html>
"""

soup = BeautifulSoup(html_content, 'html.parser')

# Find the script tag containing the data
script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
if script_tag:
    # Extract JSON data from the script tag
    json_data = script_tag.contents[0]
    
    # Convert JSON data to Python dictionary
    data_dict = json.loads(json_data)

    # Extract the relevant information
    if 'props' in data_dict and 'pageProps' in data_dict['props']:
        answers = data_dict['props']['pageProps']['answers']
        
        # Save data to CSV file
        with open('connections_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Description', 'Words', 'Color'])
            
            for answer in answers:
                description = answer['description']
                words = ', '.join(answer['words'])  # Convert list to comma-separated string
                color = answer['color']
                
                writer.writerow([description, words, color])

In [10]:
#scrapes all the code from the archive website

import csv
import json
import requests
from bs4 import BeautifulSoup

def scrape_connections_puzzle(puzzle_number):
    url = f"https://connections.swellgarfo.com/nyt/{puzzle_number}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
        if script_tag:
            json_data = script_tag.contents[0]
            data_dict = json.loads(json_data)
            if 'props' in data_dict and 'pageProps' in data_dict['props']:
                answers = data_dict['props']['pageProps']['answers']
                return answers
    return None

# Create or open the CSV file in append mode
with open('connections_all_puzzles_data.csv', 'a', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)

    # Iterate through puzzles 1 to 274
    for puzzle_number in range(1, 275):
        puzzle_data = scrape_connections_puzzle(puzzle_number)
        if puzzle_data:
            for puzzle in puzzle_data:
                description = puzzle['description']
                words = ', '.join(puzzle['words'])
                color = puzzle['color']
                writer.writerow([puzzle_number, description, words, color])
